### Authors: Axel & Tristan


In [1]:
using Flux,CairoMakie,Zygote,LinearAlgebra,IterTools

In [2]:
function node_count(model)
    """Function checks how many nodes are in each layer
       Including  the input, ouput and hidden layers"""
    param(x)  = Flux.params(model)[x]
    n = []
    for i = 1:length(Flux.params(model))
        if i%2 != 0 # Check all weights in θ 
            ni = size(param(i))[2]
            push!(n,ni)                 
        end                             
    end

    """We also have to check the output layer specifically.  
       This is because there is not weight in θ associated with the output layer"""
    push!(n, length(param(length(Flux.params(model)))))
    return n
end


function norm_params(model)
    θ(x)  = Flux.params(model)[x]
    nNodes = node_count(model)

    i = 1
    for n = 1:length(nNodes)-1
        ni = nNodes[n]

        θ(i) .= θ(i) * 1/sqrt(ni)
        i += 1
        θ(i) .= θ(i) * 1/sqrt(ni)
        i += 1

    end
end


norm_params (generic function with 1 method)

In [294]:
# x1 = [0.4]
# x2 = [1.1]
# n1 = 10


# model = Chain(Dense(1,n1,sigmoid),Dense(n1,1))|>f64



# norm_params(model)

# ∇_x1 = gradient(()->model(x1)[1],Flux.params(model))
# ∇_x2 = gradient(()->model(x2)[1],Flux.params(model))

# n = 2
# m = 2
# K = zeros(m,n)
# K[1,1] = dot(∇_x1,∇_x1)
# K[2,2] = dot(∇_x2,∇_x2)
# K[1,2] = dot(∇_x1,∇_x2)
# K[2,1] = dot(∇_x2,∇_x1)

# K

# λ = eigen(K).values



In [3]:
x1 = [0.4]
x2 = [1.1]
n1 = 1000


model = Chain(Dense(1,n1,sigmoid),Dense(n1,1))
norm_params(model)


xR = range(-1,1,step=0.1)
xR2 = range(-1,1,step=0.1)
xn = hcat(xR,xR2)
∇_xn = []
Q = length(model)
N = length(xn)
for xi = 1:N
    ∇_i = gradient(()->model(xn[:,xi])[1],Flux.params(model))
    push!(∇_xn, ∇_i)
end

Kernel = zeros(N,N)
for m = 1:N
    for n = 1:N
    Kernel[m,n] = dot(∇_xn[m],∇_xn[n])
    end
end






Kernel
λ_t = eigen(Kernel).values


DimensionMismatch: DimensionMismatch: layer Dense(1 => 1000, σ) expects size(input, 1) == 1, but got 21-element Vector{Float64}

In [4]:


# for ∇ in ∇_x1
#     println(∇)
# end

# ∇_x1[Flux.params(model)[4]]


P = length(Flux.params(model))
Ng = length(∇_xn)
∇_matrix = []
for i = 1:Ng
    ∇_col = []
    for j = 1:P-1
        param_grad = ∇_xn[i][Flux.params(model)[j]]
        push!(∇_col,param_grad)
    end
    push!(∇_matrix,∇_col)
end

∇_matrix

Any[]

In [355]:
# Function to flatten
function flatten_gradients(grad_components)
    flat_grad = Float64[]
    for comp in grad_components
        # Flatten
        append!(flat_grad, comp[:])
    end
    return flat_grad
end


flat_grads = [flatten_gradients(∇) for ∇ in ∇_matrix]

flat_grads[1]

# Kernel = zeros(Float64, length(flat_grads), length(flat_grads))
# for i in 1:length(flat_grads)
#     for j in 1:length(flat_grads)
#         Kernel[i, j] = flat_grads[i]'*flat_grads[j]
#     end
# end

# Kernel

# λ = eigen(Kernel).values


300000-element Vector{Float64}:
  1.5325591107284708e-7
 -5.743238489230862e-6
  4.017280843982007e-6
  3.196689931428409e-6
  1.827846176638559e-6
  7.267039592306901e-8
 -4.961860668117879e-6
 -4.073514446645277e-6
  2.33674450100807e-6
 -5.20527009939542e-6
  ⋮
  0.49953946471214294
  0.5010673403739929
  0.5017957091331482
  0.49919593334198
  0.4994317591190338
  0.5015895962715149
  0.5012345314025879
  0.5000563859939575
  0.5017050504684448

In [353]:
function check_dim(x)
    """This function checks the appropriate  dimensions of input data"""
    if isa(x, Matrix)
        return size(x, 2)  # Returns the number of columns (width) of the matrix
    elseif isa(x, Vector)
        return 1  # Return 1 if it's a column vector
    else
        type = typeof(x)
        error("Input data type: $type is neither a matrix or column vector")
    end
end

function jac(model, x, f,param)
    """Gets the jacobian of a specific parameter"""
    jaco(f) = Flux.jacobian(() -> model(x)[f],Flux.params(model))
    return jaco(f)[Flux.params(model)[param]]
end

function Df(model, x)
    # x: single datapoint
    m = length(model(x))

    # Total amount of θ exluding final bias
    total_amount_of_θ = sum(length, Flux.params(model))  - length(Flux.params(model)[length(Flux.params(model))])

    # Skilgreini empty jacobian matrix
    Jacob = zeros(total_amount_of_θ,m)

    for func_i = 1:m
        current_col = Vector{Float64}(undef, 0) # Preallocate memory
        for param_i = 1:length(Flux.params(model)) - 1
            jac_vec = jac(model, x, func_i, param_i)[:]
            current_col = vcat(current_col, jac_vec) # Concatenate vectors
        end
        for k = 1:total_amount_of_θ 
            Jacob[k, func_i] = current_col[k]
        end
    end

    return Jacob # Þetta er Df fylkið í bilblíunni
end

Df (generic function with 1 method)

In [356]:
J = Df(model,xn)

J[:,1] == flat_grads[1]

true